In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.5.0'

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    #df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [5]:
print(df_january.shape, df_february.shape)

(3066766, 19) (2913955, 19)


## Homework

## Punto 1

In [6]:
df_january.shape[1]

19

In [7]:
df_january.tpep_dropoff_datetime = pd.to_datetime(df_january.tpep_dropoff_datetime)
df_january.tpep_pickup_datetime = pd.to_datetime(df_january.tpep_pickup_datetime)

df_february.tpep_dropoff_datetime = pd.to_datetime(df_february.tpep_dropoff_datetime)
df_february.tpep_pickup_datetime = pd.to_datetime(df_february.tpep_pickup_datetime)

df_january['duration'] = df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime
df_january.duration = df_january.duration.apply(lambda td: td.total_seconds() / 60)


df_february['duration'] = df_february.tpep_dropoff_datetime - df_february.tpep_pickup_datetime
df_february.duration = df_february.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
# POINT 2
df_january.duration.std()

42.594351241920904

In [9]:
# Calcula la desviación estándar de la duración de los viajes
std_duration = df_january['duration'].std()
print(f'Standard Deviation of trip duration in January: {std_duration}')

Standard Deviation of trip duration in January: 42.594351241920904


In [10]:
# POINT 3
df_january_filtered = df_january[(df_january.duration >= 1) & (df_january.duration <= 60)]
df_february_filtered = df_february[(df_february.duration >= 1) & (df_february.duration <= 60)]
print(df_january_filtered.shape, df_february_filtered.shape)

(3009173, 20) (2855951, 20)


In [11]:
print(df_january.shape, df_february.shape)

(3066766, 20) (2913955, 20)


In [12]:
print(f"Left: {round(df_january_filtered.shape[0]/df_january.shape[0] * 100)}% of the records")

Left: 98% of the records


In [13]:
df_january = df_january[(df_january.duration >= 1) & (df_january.duration <= 60)]
df_february = df_february[(df_february.duration >= 1) & (df_february.duration <= 60)]

In [14]:
df_january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [15]:
# POINT 4

In [16]:
from sklearn.feature_extraction import DictVectorizer

/home/codespace/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
# Paso 1: Asegúrate de que las columnas PULocationID y DOLocationID sean de tipo str
df_january['PULocationID'] = df_january['PULocationID'].astype(str)
df_january['DOLocationID'] = df_january['DOLocationID'].astype(str)

# Convertir el dataframe en una lista de diccionarios, usando solo PULocationID y DOLocationID
dicts = df_january[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Paso 2: Ajustar el DictVectorizer usando una representación dispersa
dv = DictVectorizer(sparse=True)
X_sparse = dv.fit_transform(dicts)

# Paso 3: Obtener la dimensionalidad de la matriz de características
print(f'Dimensionality of the matrix: {X_sparse.shape[1]}')

Dimensionality of the matrix: 515


In [18]:
# POINT 5
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Usar la duración de los viajes como la variable objetivo
y = df_january['duration'].values
X = dv.fit_transform(dicts)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X, y)

# Predecir las duraciones usando el modelo entrenado
y_pred = model.predict(X)

# Calcular el RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f'RMSE on training data: {rmse}')

RMSE on training data: 7.649261027806873


In [20]:
# POINT 6
# Paso 1: Asegúrate de que las columnas PULocationID y DOLocationID sean de tipo str
df_february['PULocationID'] = df_february['PULocationID'].astype(str)
df_february['DOLocationID'] = df_february['DOLocationID'].astype(str)

dicts_february = df_february[['PULocationID', 'DOLocationID']].to_dict(orient='records')
# Usar la duración de los viajes como la variable objetivo
y_test = df_january['duration'].values
X_test = dv.transform(dicts_february)


# Predecir las duraciones usando el modelo entrenado
y_test_pred = model.predict(X_test)

# Calcular el RMSE
rmse_validation = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'RMSE on training data: {rmse_validation}')

ValueError: Found input variables with inconsistent numbers of samples: [3009173, 2855951]